 <center><font size="8">Model Evaluation</font></center>

In [2]:
# Import libraries
import os
import copy
from tqdm.notebook import tqdm
import pprint
from ipywidgets import interact, GridspecLayout, interactive, interact_manual,Layout
import ipywidgets as widgets
from IPython.display import display
import torch
from src.constants import DATA_PATH, MASTER_THESIS_DIR, TRAINING_CONFIG_PATH, SAVED_MODELS_BASE_PATH,SUPERVISED_CONFIG_PATH
import pandas as pd
from src.data_loader.data_set import Data_Set
# from src.experiments.utils import get_experiement_args, process_experiment_args
from src.models.baseline_model import BaselineModel
from src.utils import get_console_logger, read_json
from src.visualization.visualize import plot_hand
from torch.utils.data import DataLoader
from torchvision import transforms
from easydict import EasyDict as edict
import matplotlib.pyplot as plt
from src.data_loader.utils import convert_2_5D_to_3D, convert_to_2_5D
from src.experiments.evaluation_utils import calculate_epe_statistics, evaluate,get_predictions_and_ground_truth, cal_auc_joints, get_pck_curves
from src.data_loader.utils import get_train_val_split
from src.models.supervised_head_model import SupervisedHead
from IPython.display import clear_output
from src.experiments.utils import restore_model
import numpy as np
from src.models.denoised_baseline import DenoisedBaselineModel

In [3]:
# global variables:
results = None
model = None
x=y=None
params = {"ytick.color" : "w",
          "xtick.color" : "w",
          "axes.labelcolor" : "w",
          "axes.edgecolor" : "w",
         "text.color":"w"}
plt.rcParams.update(params)

In [4]:
# Load configuration
model_config = edict(read_json(SUPERVISED_CONFIG_PATH))
joints_mapping = {
    v: k
    for k, v in read_json(f"{MASTER_THESIS_DIR}/src/data_loader/joint_mapping.json")[
        "ait"
    ].items()
}
train_param = edict(
    read_json(f"{MASTER_THESIS_DIR}/src/experiments/config/training_config.json")
)
train_data = None


@interact(
    source=widgets.Dropdown(
        options=["interhand", "freihand"],
        value="freihand",
        description="source",
        disabled=False,
    )
)
def select_data(source):
    global train_data
    train_data = Data_Set(
        config=train_param,
        transform=transforms.ToTensor(),
        train_set=False,
        experiment_type="supervised",
        source=source,
    )

interactive(children=(Dropdown(description='source', index=1, options=('interhand', 'freihand'), value='freiha…

In [5]:
def select_augmentations(idx,random_crop,crop,color_jitter,cut_out, resize, color_drop,gaussian_blur,rotate,sobel_filter,gaussian_noise, training):
    global train_data
    train_param.augmentation_flags.random_crop = random_crop
    train_param.augmentation_flags.crop = crop
    train_param.augmentation_flags.color_jitter = color_jitter
    train_param.augmentation_flags.cut_out = cut_out
    train_param.augmentation_flags.resize = resize
    train_param.augmentation_flags.color_drop = color_drop
    train_param.augmentation_flags.gaussian_blur = gaussian_blur
    train_param.augmentation_flags.rotate = rotate
    train_param.augmentation_flags.sobel_filter = sobel_filter
    train_param.augmentation_flags.gaussian_noise = gaussian_noise
    train_data.augmenter = train_data.get_sample_augmenter(
        train_param.augmentation_params, train_param.augmentation_flags
    )
    train_data.is_training(training)
    sample = train_data[idx]
    joints = sample["joints"]
    img = torch.flip(sample["image"],(0,))
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    plt.imshow(transforms.ToPILImage()(img))
    plt.xticks([])
    plt.yticks([])
    plt.title("Augmentated sample",fontsize=20)
    plot_hand(ax, joints)
    return ax

font_size =4
augmentation_updated_interact = interactive(select_augmentations,
    idx = widgets.IntSlider(min=1, max=3000, step=5, value=3),
    random_crop=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Random crop</font>"),
    crop=widgets.Checkbox(value=True, description=f"<font size='{font_size}'>Crop</font>"),
    color_jitter=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Color jitter</font>"),
    cut_out=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Cut out</font>"),
    resize=widgets.Checkbox(value=True, description=f"<font size='{font_size}'>Resize</font>"),
    color_drop=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Color drop</font>"),
    rotate=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Rotate</font>"),
    gaussian_blur=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Blur(gaussian)</font>"),
    gaussian_noise=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Noise(gaussian)</font>"),
    sobel_filter=widgets.Checkbox(value=False, description=f"<font size='{font_size}'>sobel_filter</font>"),
    training = widgets.Checkbox(value=False, description=f"<font size='{font_size}'>Train set</font>"),)
augmentation_checkboxes = augmentation_updated_interact.children[:-1]
augmentation_grid = GridspecLayout(6,4, height='400px')
for i in range(5):
    for j in range(2):
        augmentation_grid[i,j] = augmentation_checkboxes[i*2+j+1]
augmentation_grid[5,2:] = augmentation_updated_interact.children[0]
augmentation_grid[2,3] = augmentation_updated_interact.children[-2]
augmentation_grid[:5,2:3] = augmentation_updated_interact.children[-1]
# augmentation_grid[:5,3:].layout.height = 'auto'
augmentation_updated_interact.children[0].description = f"<font size='{font_size}'>Index</font>"

<font size="6">Select augmentations</font>

In [7]:
display(augmentation_grid)

GridspecLayout(children=(Checkbox(value=False, description="<font size='4'>Random crop</font>", layout=Layout(…

<font size="6">Load Model</font>

In [1]:
def load_model(key, denoiser):
    global model
    print(f"Loading latest checkpoint of {key}")
    try:
        model_config.saved_model_name = key
        model_config.checkpoint = ""
        # semi supervised model
        if denoiser:
            model = DenoisedBaselineModel(model_config)
        else:
            model = SupervisedHead(
                model_config,
            )
        model = restore_model(model, key, "")
        model.eval()
        print("Model loaded successfully!")
    except Exception as e:
        print(e)
        model = None
        print(f"Experiment {key} not found !")


def calc_results():
    global x, y
    global results
    global model
    global train_data
    val_data = train_data
    if model is not None:
        clear_output(wait=True)
        print(
            f"Calculating results on validation set formed with seed {train_param.seed}"
        )
        prediction_dict = get_predictions_and_ground_truth(
            model, val_data, num_workers=8, batch_size=128
        )
        epe_2D = calculate_epe_statistics(
            prediction_dict["predictions"], prediction_dict["ground_truth"], dim=2
        )
        epe_3D = calculate_epe_statistics(
            prediction_dict["predictions_3d"], prediction_dict["ground_truth_3d"], dim=3
        )
        epe_3D_recreated = calculate_epe_statistics(
            prediction_dict["predictions_3d"],
            prediction_dict["ground_truth_recreated_3d"],
            dim=3,
        )
        epe_3D__gt_vs_3D_recreated = calculate_epe_statistics(
            prediction_dict["ground_truth_3d"],
            prediction_dict["ground_truth_recreated_3d"],
            dim=3,
        )
        if hasattr(model, "denoiser"):
            epe_3D_gt_vs_denoised = calculate_epe_statistics(
                prediction_dict["ground_truth_3d"],
                prediction_dict["predictions_3d_denoised"],
                dim=3,
            )
            auc_denoised = np.mean(cal_auc_joints(epe_3D_gt_vs_denoised["eucledian_dist"]))
            denoised_results = {
            "Mean_EPE_3D_denoised": epe_3D_gt_vs_denoised["mean"].cpu(),
            "Median_EPE_3D_denoised": epe_3D_gt_vs_denoised["median"].cpu(),
            "auc_denoised": auc_denoised,
        }
        else:
            denoised_results ={}
        eucledian_dist = epe_3D["eucledian_dist"]
        y, x = get_pck_curves(eucledian_dist, per_joint=True)
        auc = cal_auc_joints(eucledian_dist, per_joint=True)
        results = {**{
            "Mean_EPE_2D": epe_2D["mean"].cpu().numpy(),
            "Median_EPE_2D": epe_2D["median"].cpu().numpy(),
            "Mean_EPE_3D": epe_3D["mean"].cpu().numpy(),
            "Median_EPE_3D": epe_3D["median"].cpu().numpy(),
            "Mean_EPE_3D_R": epe_3D_recreated["mean"].cpu().numpy(),
            "Median_EPE_3D_R": epe_3D_recreated["median"].cpu().numpy(),
            "Mean_EPE_3D_R_v_3D": epe_3D__gt_vs_3D_recreated["mean"].cpu().numpy(),
            "Median_EPE_3D_R_V_3D": epe_3D__gt_vs_3D_recreated["median"].cpu().numpy(),
            "AUC": np.mean(auc),
        },**denoised_results}
        clear_output(wait=True)
        results_df = pd.DataFrame.from_dict(
            {k: ["{:.3f}".format(float(v))] for k, v in results.items()}
        ).T
        results_df = results_df.rename(columns={0: "value"})
        results_df.index.name = "Metric"
        results_df = results_df.reset_index()
        del prediction_dict
        return display(results_df)
    else:
        return "No model selected!", "Nothing to plot in AUC"


def plot_results():
    global x, y
    global results
    if results is not None:
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111)
        ax.set_facecolor("black")
        plt.plot(x, np.mean(y, axis=0), color="green", linewidth="1", alpha=1)
        plt.xlabel("error in mm", fontsize=12)
        plt.ylabel("Ratio of points below the error", fontsize=12)
        plt.legend(["Average PCK curve"], facecolor="black")
        plt.title(f"AUC: {results['AUC']}")
        return ax
    else:
        print("Calculate Results first!")


load_calculate_button_grid = GridspecLayout(3, 4, height="100px")
load_calculate_display_grid = GridspecLayout(5, 4, height="300px")
display(load_calculate_button_grid)
display(load_calculate_display_grid)

load_model_interact = interactive(
    load_model,
    {"manual": True},
    key=widgets.Text(
        value="1c4fda5d61ef4d5b93ced45e4e86d795",
        description=f"<font size='{font_size}'>Experiment</font>",
    ),
    denoiser=widgets.Checkbox(
        value=False, description=f"<font size='{font_size}'>Denoiser</font>"
    ),
)
plot_pck_interact = interactive(plot_results, {"manual": True})
calculate_result_interact = interactive(calc_results, {"manual": True})
experiment_key_textbox = load_model_interact.children[0]
load_model_button = load_model_interact.children[2]
load_model_checkbox = load_model_interact.children[1]
load_model_button.description = "Load model"
plot_result_button = plot_pck_interact.children[0]
plot_result_button.description = "Plot pck curve"
calculate_result_interact.children[0].description = "Results"

load_calculate_button_grid[0, 0] = experiment_key_textbox
load_calculate_button_grid[1,0] = load_model_checkbox
load_calculate_button_grid[0, 1] = load_model_button
load_calculate_display_grid[0:, 0] = load_model_interact.children[-1]
load_calculate_button_grid[0, 2] = calculate_result_interact.children[0]
load_calculate_button_grid[0, 3] = plot_result_button
load_calculate_display_grid[0:, 2:3] = calculate_result_interact.children[-1]
load_calculate_display_grid[0:, 3:] = plot_pck_interact.children[-1]
# load_calculate_result_grid[1:,0] = load_model_interact.children[-1]
# load_calculate_result_grid[1:,1:] = calculate_result_interact.widget.children[-1][

NameError: name 'GridspecLayout' is not defined

<font size="6">Visualize results</font>

In [95]:
def visualize1(idx, plot_gt, plot_pred):
    global model, train_data
    if model is not None:
        model.to(torch.device('cpu'))
        model.eval()
        sample = train_data[idx]
        joints = sample["joints"]
        img = sample["image"]
        img_input = img.view(([1]+list(img.shape)))
        prediction = model(img_input).view(21,3).detach().numpy()
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111)
        plt.imshow(transforms.ToPILImage()(img))
        ax.set_facecolor('black')
        if plot_gt:
           plot_hand(ax, joints)
        if plot_pred:
            plot_hand(ax, prediction,linestyle=':' )
        plt.xticks([])
        plt.yticks([])
        return ax
    else:
        print("Model is not defined!")

visualize_predictions = interactive(visualize1, idx = widgets.IntSlider(min=1, max=3000, step=5, value=3),plot_gt= widgets.Checkbox(value=True, description=f"<font size='{font_size}'>Ground truth</font>"),
         plot_pred= widgets.Checkbox(value=True, description=f"<font size='{font_size}'>Prediction</font>")
           )
visualize_grid = GridspecLayout(4,3, height="400px")
visualize_grid[1,0] = visualize_predictions.children[2]
visualize_grid[2,0] = visualize_predictions.children[1]
visualize_grid[:3,1:] = visualize_predictions.children[-1]
visualize_grid[3,1] = visualize_predictions.children[0]
visualize_predictions.children[0].description = f"<font size='{font_size}'>Index</font>"
display(visualize_grid)

GridspecLayout(children=(Checkbox(value=True, description="<font size='4'>Prediction</font>", layout=Layout(gr…